In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746532 sha256=e4178afbb74c1a63d57c3344c5d6434fe520bf006c0788d95de21cc58e1bf27b
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [ ]:
!mpirun -n 2 --allow-run-as-root --oversubscribe echo "hello"

hello
hello


In [ ]:
%%writefile testmpi.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print(f"I am {rank} from {size}")

if rank == 0:
    print("I am doing job 0")
elif rank == 1:
    print("I am doing job 1")

Overwriting testmpi.py


In [ ]:
!mpirun -n 4 --oversubscribe --allow-run-as-root python testmpi.py

I am 3 from 4
I am 2 from 4
I am 1 from 4
I am 0 from 4
I am doing job 0
I am doing job 1


In [ ]:
%%writefile testmpi.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    a = ['hello world', 10, [1,2], {'a': 5}]
    comm.send(a, dest=1, tag=0)
elif rank == 1:
    a = comm.recv(source=0, tag=0)
    print(a)

Overwriting testmpi.py


In [ ]:
!mpirun -n 2 --oversubscribe --allow-run-as-root python testmpi.py

In [ ]:
%%writefile testnumpy.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    nofs = 16
    a = np.linspace(0, 16, nofs, endpoint=False, dtype=np.int32)
    comm.send(nofs, dest=1, tag=0)
    comm.Send([a[2:6], 4, MPI.INT], dest=1, tag=0)
elif rank == 1:
    nofs2 = comm.recv(source=0, tag=0)
    a = np.zeros(nofs2, dtype=np.int32)
    comm.Recv([a[0:4], 4, MPI.INT], source=0, tag=0)
    print(a)

Overwriting testnumpy.py


In [ ]:
!mpirun -n 2 --oversubscribe --allow-run-as-root python testnumpy.py

[2 3 4 5 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
%%writefile pi.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

global_N = 2**20

my_N = global_N // size
remainder = global_N % size

if (rank < remainder):
    my_N = my_N + 1

my_count = 0
for n in range(my_N):
    x = np.random.rand()
    y = np.random.rand()
    if (x**2 + y**2) < 1:
        my_count = my_count + 1

my_pi = np.array([4 * my_count / my_N / size], dtype=np.float64)

global_pi = None
if rank == 0:
    global_pi = np.array([0], dtype=np.float64)

comm.Reduce(my_pi, global_pi, op=MPI.SUM, root=0)

print(global_pi )

Overwriting pi.py


In [ ]:
!time mpirun -n 2 --oversubscribe --allow-run-as-root python pi.py

None
[3.14027023]

real	0m1.939s
user	0m2.831s
sys	0m0.255s


In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [ ]:
Comm.Reduce(sendbuf, recvbuf, op=MPI.SUM, root=0)